# Microphysics Simulation
Now that you learned how to run a simulation with fuse from a geant4 file up to `raw_records` we will take a look the the data that fuse can produce in the intermediate simulation steps fucusing on the microphysics simulation. This part of the XENONnT simulation chain was formerly done using the `epix` software.

## Imports & Simulation Context
Just like in the Getting_Started notebook we will first do our necessary imports and then set up the full chain simulation context.  

In [5]:
import fuse
import matplotlib.pyplot as plt
import numpy as np
from straxen import URLConfig

In [6]:
url_string = 'simple_load://resource://format://fax_config_nt_sr0_v4.json?&fmt=json'
config = URLConfig.evaluate_dry(url_string)

In [7]:
st = fuse.context.full_chain_context(out_dir = "./fuse_data",
                                     config = config)

st.set_config({"path": "/project2/lgrandi/xenonnt/simulations/testing",
               "file_name": "pmt_neutrons_100.root",
               "entry_stop": 10,
               })

run_number = "00000"

/dali/lgrandi/xenonnt/software/cutax/v1.15.1/cutax/contexts.py:107: UserWarning: WARNING! You are using SR0_V3 simulation context! Consider updating to a newer/pinned version (SR0_V4)!
  warnings.warn(f'WARNING! You are using {faxconf_version.upper()} simulation context!'
/dali/lgrandi/xenonnt/software/cutax/v1.15.1/cutax/contexts.py:132: UserWarning: WARNING! [ FAX_CONFIG_NT_SR0_V3.JSON ] does not contain "field_distortion_correction_map" key!
  warnings.warn(f'WARNING! [ {fax_conf.upper()} ] does not contain "{fax_config_fdc_key}" key!')


## Running the Simulation
Just like in the Getting_Started notebook we will run the simulation using the `st.make(run_number, "target")` function from strax. This time we will tell fuse explicitly to save the intermediate simulation output. 

### Reading the root file and assigning the cluster index
Before we do some simulation and start to calculate stuff, we need to read in the root file from Geant4 and convert it into a data format that can be handled by strax. This step is done in the `ChunkInput` plugin and the result can be accessed using target `geant4_interactions`. This plugin used uproot to open the root file and then converts it into numpy arrays. Each event is then assigned a time based on the config option `source_rate`. The plugin will cut delayed interaction and devide the data into chunks if necessary. 

In the next simulation step we will give each interaction a cluster index. This is done by the `FindCluster` plugin. First, all interactions are grouped by time and then a DBSCAN algorithm is used to spacially cluster the interactions. The output of this plugin can be accessed using target `cluster_index` and can be loaded along with the `geant4_interactions`.

In [17]:
st.make(run_number, "geant4_interactions")
st.make(run_number, "cluster_index")

geant4_interactions = st.get_df(run_number,["geant4_interactions", "cluster_index"])

Loading plugins: |          | 0.00 % [00:00<?]

Lets take a look at the data:

In [18]:
geant4_interactions.head(10)

,cluster_ids,time,endtime,x,y,z,t,ed,type,trackid,parenttype,parentid,creaproc,edproc,evtid,x_pri,y_pri,z_pri
0,0,2185013524,2185013524,-20.890713,-51.367107,-1.464800,21.859256,5.101432,Xe131,2,neutron,1,hadElastic,ionIoni,0,-49.813919,0.516251,9.241811
1,1,2704673279,2704673279,-25.334255,-4.493484,-21.541744,8.586755,2.472048,Xe131,102,neutron,2,hadElastic,ionIoni,1,-44.154747,2.803934,7.933488
2,2,2704673279,2704673279,-25.548460,-4.034955,-21.613764,8.223299,72.487770,Xe129,3,neutron,1,neutronIne,ionIoni,1,-44.154747,2.803934,7.933488
3,3,2704673280,2704673280,-10.058850,14.729630,-38.732780,9.384739,0.036780,e-,93,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
4,3,2704673280,2704673280,-10.058850,14.729630,-38.732780,9.384740,0.022620,e-,94,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
5,3,2704673280,2704673280,-10.058849,14.729630,-38.732780,9.384740,0.012420,e-,95,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
6,3,2704673280,2704673280,-10.058850,14.729630,-38.732780,9.384740,0.033880,e-,96,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
7,3,2704673280,2704673280,-10.058849,14.729630,-38.732780,9.384739,0.012420,e-,97,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
8,3,2704673280,2704673280,-10.058849,14.729631,-38.732780,9.384739,0.044470,e-,98,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488
9,3,2704673280,2704673280,-10.058849,14.729630,-38.732780,9.384739,0.044470,e-,99,gamma,80,phot,msc,1,-44.154747,2.803934,7.933488


### Clustering and Volume Cuts

In the next step, all interactions with the same `cluster_ids` are merged. The energy of the interactions is summed up and the position and time is calculated as the weighted average of the positions of the individual interactions. The interaction type of the cluster is determined either by the interaction with the highest energy or by the first interaction in the cluster. The interaction type is later used to choose the correct emmision model. 

Following the clustering, the `VolumePlugin`s `XENONnT_TPC` and `XENONnT_BelowCathode` are used to select only interactions in these detector regions and assigne them the corresponding xenon density and decide if S2s can be created in this volume. The interactions are merged via the `VolumesMerger` plugin and the result can be accessed using target `interactions_in_roi`. `roi` stands for region of interest and and indicates a physical volume in the detector. 

In [19]:
st.make(run_number, "clustered_interactions")
st.make(run_number, "interactions_in_roi")

### Electric Field and Emission Model

The aim of this simulation part is to model the scintillation and ionization processes at the interaction site. First we need to estimate the electric field strength at the interaction position. This is done in the `ElectricField` plugin using a simulated field map. The field values can be accessed using the target `electric_field_values`. Next we can estimate the number of produced electrons and photons using an emission model. The default implementation of fuse uses the `NestYields` plugin where `nestpy` is used. fuse also provides alternative plugins where the yields are calculated using BBF or a beta response model. These plugins should only be used if you know what you are doing. The result of the emission model can be accessed using the target `quanta`. 

In [ ]:
st.make(run_number, "electric_field_values")
st.make(run_number, "quanta")

Finally we can collect the simulation results of the last few steps using the `MicroPhysicsSummary` plugin. This plugin is a `strax.MergeOnlyPlugin` and does not do any calculations. It just merges the results of the previous plugins and can be accessed using the target `microphysics_summary`.

In [20]:
st.make(run_number, "microphysics_summary")

microphysics_summary = st.get_df(run_number,["microphysics_summary"])

/opt/XENONnT/anaconda/envs/XENONnT_2023.07.1/lib/python3.9/site-packages/numpy/lib/function_base.py:2387: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


Loading ['microphysics_summary']: |          | 0.00 % [00:00<?]

In [22]:
microphysics_summary.head()

,e_field,time,endtime,x,y,z,ed,nestid,A,Z,evtid,x_pri,y_pri,z_pri,xe_density,vol_id,create_S2,photons,electrons,excitons
0,35,2185013524,2185013524,-20.890713,-51.367104,-1.464800,5.101432,0,0,0,0,-49.813919,0.516251,9.241811,2.862,1,True,37.0,36.0,30.0
1,25,2704673279,2704673279,-25.334255,-4.493484,-21.541744,2.472048,0,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,12.0,15.0,9.0
2,25,2704673279,2704673279,-25.548460,-4.034955,-21.613764,72.487770,0,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,1023.0,181.0,620.0
3,27,2704673279,2704673279,-36.511883,-21.699745,-9.259168,4.774700,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,190.0,148.0,17.0
4,27,2704673279,2704673279,-36.386059,-21.583389,-9.211907,34.503632,8,0,0,1,-44.154747,2.803934,7.933488,2.862,1,True,2012.0,496.0,391.0
